In [48]:
from utils import *

In [49]:
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication(tenant_id="c643d8c8-4abb-47ba-808c-bf73f60161f1")

# Configure workspace
ws = Workspace.from_config(auth=interactive_auth)

In [50]:
from azureml.core.model import InferenceConfig

env = get_env(name="sentiment-analysis", packages=["inference-schema", "tensorflow", "transformers"])

inference_config = InferenceConfig(
    environment=env,
    source_directory="code",
    entry_script="sentiment_analysis_batch.py",
)

In [51]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=0.5, memory_gb=1, auth_enabled=True
)

In [52]:
from azureml.core import Model

service = Model.deploy(
    ws,
    "sentiment-analysis-batch",
    [],
    inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-01-02 17:55:05+00:00 Creating Container Registry if not exists.
2022-01-02 17:55:05+00:00 Registering the environment.
2022-01-02 17:55:05+00:00 Use the existing image.
2022-01-02 17:55:05+00:00 Generating deployment configuration.
2022-01-02 17:55:06+00:00 Submitting deployment to compute.
2022-01-02 17:55:08+00:00 Checking the status of deployment sentiment-analysis-batch..
2022-01-02 17:58:10+00:00 Checking the status of inference endpoint sentiment-analysis-batch.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [53]:
print(service.get_logs())

2022-01-02T17:57:25,031546100+00:00 - iot-server/run 
2022-01-02T17:57:25,033956200+00:00 - rsyslog/run 
2022-01-02T17:57:25,032872900+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2022-01-02T17:57:25,094030500+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-01-02T17:57:25,612636600+00:00 - iot-server/finish 1 0
2022-01-02T17:57:25,621016200+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (68)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 94
2022-01-02 17:57:28.117313: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /azureml-envs/azureml_0b48319fab02ae60c0ce02977a7216a1/lib:/azureml-envs/azureml_0b48319fab02ae60c0ce02977a721

In [54]:
import requests
import json
from azureml.core import Webservice

service = Webservice(workspace=ws, name="sentiment-analysis-batch")
scoring_uri = service.scoring_uri

# If the service is authenticated, set the key or token
key, _ = service.get_keys()

# Set the appropriate headers
headers = {"Content-Type": "application/json"}
headers["Authorization"] = f"Bearer {key}"

data = {
    "Inputs": {
        "data": [
            "My ML skills are quite good.",
            "I didn't have good experience with ML."
        ]
    }
}
data = json.dumps(data)
resp = requests.post(scoring_uri, data=data, headers=headers)
print(resp.text)

[0.9998338222503662, -0.9997455477714539]


In [55]:
scoring_uri

'http://0680950f-e6cd-4944-91cf-cf6c4f7a98de.westus2.azurecontainer.io/score'

In [56]:
# service.delete()